# Financial Econometrics I: Homework 2

Team Member:

Lin Zhang : 15845542@fsv.cuni.cz

Weiwei Qu : 51014941@fsv.cuni.cz

# Problem 1

### Select 100 random symbols from the symbols.csv file, i.e. using a function in R generate 100 random integers from interval f1,..., 377g, use set.seed('your SIS number') to generate these numbers(e.g. set.seed(26830192)). Then select the Tickers from rows with indices equal to the numbers you generated, and download the prices of stocks denoted by these Tickers for the period 01/2019 - 12/2021. Please restrict the period immediately when downloading the data, it should get downloaded more quickly then.

##### 

* Setup environment and load libraries

In [ ]:
# Setup environment
Sys.setenv(LANG = "en")
Sys.setlocale("LC_TIME", "English")
#options(warn = -1)  # suppressing warnings

if (!require(quantmod)) install.packages('quantmod')
if (!require(fGarch)) install.packages('fGarch')
if (!require(repr)) install.packages('repr')
if (!require(rugarch)) install.packages('rugarch')
if (!require(aTSA)) install.packages('aTSA')
if (!require(tseries)) install.packages('tseries')
if (!require(fDMA)) install.packages('fDMA')
if (!require(ggplot2)) install.packages('ggplot2')
if (!require(ggthemes)) install.packages('ggthemes')


library(quantmod)
library(fGarch)
library(repr)
library(forecast)
library(aTSA)
library(rugarch)
library(tseries)
library(fDMA)
library(ggplot2)
library(ggthemes) 

options(repr.plot.width = 8, repr.plot.height = 6)

* Select 100 random symbols from requirement file

In [ ]:
# read 377 stock symbols from file
smbs<- as.character(read.csv("symbols2.csv")[,2])
length(smbs)
smbs

In [ ]:
set.seed(15845542) # seed for random number generation
s_symbols<-sample(smbs,100) # generate 100 random firms
length(s_symbols)
s_symbols

* Download closing price data

In [ ]:
#download data
stock_price <- lapply(s_symbols, function(y)
{
    getSymbols(y, auto.assign = FALSE,from = as.Date('2019-01-01'), to = '2021-12-31')
})
names(stock_price) <- s_symbols

# save data for further usage
#  save(stock_price, file = "1-Price.RData")
# load("1-Price.RData")

* Check the data validity（Reference the Hw1 solution from SIS)

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 6)
par(bg = "#f7f7f7")
stock_price1 <- sapply(stock_price,na.omit)
nrows <- unlist(sapply(stock_price1, nrow))
plot(nrows)

Clearly there is one stock does not have enough data set from downloading.

In [ ]:
length(stock_price1[[1]])
length(stock_price1[[43]])
tail(stock_price1[[43]])

**<span style='background:yellow'>We notied that NFX does not have enough data, it need to be removed from our portfolio.</span>**

In [ ]:
stock_price$NFX <- NULL
length(stock_price)
s_symbols <- names(stock_price)

* Clean the data set, only need Adjusted closing price for our calculations.

In [ ]:
# filter for Adjusted price
stock_close <- lapply(s_symbols, function(y){
    stock_price[[y]] <- stock_price[[y]][, paste0(y, '.Adjusted')]
})

# check all symbols used for downloading process are used, no data is missing
length(stock_price)
head(stock_close[[1]])
tail(stock_close[[1]])

###  1. Compute the logarithmic returns for the time-series of each stock in your dataset.


In [ ]:
# compute log-returns for returns
# multiple 100 to make easier observation

lrets <- lapply(stock_close, function(y){
  y <- na.omit(diff(log(y))) * 100
})
names(lrets) <- s_symbols

head(lrets[[1]])
tail(lrets[[1]])

# Save data for later usage
#  save(lrets, file = "2-Returns.RData")
# load("2-Returns.RData")

**Note: We use <span style='background:yellow'>percentage number as returns</span>, which already be multipled by 100.**

###  2. For each stocks, estimate the parameters of a GARCH(1, 1) model.

* Apply GARCH model to estimate each stock

In [ ]:
# Set parameters for GARCH(1,1) by ugarchspec()
GarchSet1 <- ugarchspec(mean.model = list(armaOrder = c(0, 0)),
                variance.model = list(garchOrder = c(1, 1)))

# Create matrix stock_coef to store estimate parameters of stocks
N <- length(lrets)  
x <- c(1:N * 4)
rown <- s_symbols
coln <- c("alpha1", "beta1", "alpha1+beta1", "omega")
stock_coef <- matrix(x, nrow = N, ncol = 4, byrow = TRUE, 
            dimnames = list(rown, coln))
l_alpha <- list()
l_beta <- list()
l_omega <- list()

# Run ugarchfit() model to all stocks
for (i in 1 : N){
    GarchFit1 <- ugarchfit(GarchSet1, lrets[[i]])
    l_alpha <- c(l_alpha, GarchFit1@fit$coef['alpha1'])
    l_beta <- c(l_beta, GarchFit1@fit$coef['beta1'])  
    l_omega <- c(l_omega, GarchFit1@fit$coef['omega'])  
}

for (i in 1 : N){
    stock_coef[i,1] <- round(l_alpha[[i]],6)
    stock_coef[i,2] <- round(l_beta[[i]],6)    
    stock_coef[i,4] <- round(l_omega[[i]],6) 
}
stock_coef[,3] <- stock_coef[,1] + stock_coef[,2]

head(stock_coef)
tail(stock_coef)

**We store those coefficient parameters of all stocks into a matrix named <span style='background:yellow'>"stock_coef"</span>.** 

In [ ]:
# Save data for later usage
#  save(stock_coef, file = "3-Coef.RData")
# load("3-Coef.RData")

### 3. You will end up with 100 sets of estimated coefficients. Plot the histogram of the 100 $\alpha_1$ coeffcients, and repeat with $\beta_1$ , and $\alpha_1+\beta_1$.   Comment on the cross-sectional variation of the $\alpha_1,\beta_1$,  and $\alpha_1+\beta_1$ and on the consequences that the values of these coeffcients have on the behaviour of the estimated volatility.

* Plot the histogram of the 100 $\alpha_1$ coeffcients, and repeat with $\beta_1$ , and $\alpha_1+\beta_1$. 

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10)
par(bg = "#f7f7f7")
par(mfrow = c(3, 1))

hist(stock_coef[,"alpha1"], n = 100, probability = TRUE, border = "white",
     col = "steelblue",xlab = NA, main = "Alpha1" )

hist(stock_coef[,"beta1"], n = 100, probability = TRUE, border = "white",
     col = "steelblue", xlab = NA, main = "Beta1" )

hist(stock_coef[,"alpha1+beta1"], n = 100, probability = TRUE, border = "white",
     col = "steelblue", xlab = NA, main = "Alpha1+Beta1" )

**Comments:  
In a GARCH(1,1) process, we have formula as**  
$$ y_t = \mu_t + a_t  $$
**$a_t$ is for volatility, $\mu_t$ is mean.**

$$ a_t = \sigma_t \epsilon_t \\
\sigma_t^2 = \omega + \alpha_1 a_{t - 1}^2 + \beta_1 \sigma_{t - 1}^2.$$



**We notice that in GRACH(1,1) model, all stocks $\alpha_1+\beta_1$ are close to 1.  
When $\beta_1$ is small, the volatility cluster is not obvious. But when $\beta_1$ is greater, because the autocorrelation of $\sigma$ is higher, the volatility spike persistence will longer, there will be a slow decay of volatility, and will show volatility clustering.   
So from the formula we could see, the volatility equation combines connection with past returns as well as autoregressive dependence.**

###  4. Find the minimum and maximum values of $\alpha_1,\beta_1$,  and $\alpha_1+\beta_1$.  Comment briefly.

* Show the minimum and maximum parameters of the portfolio

In [ ]:
apply(stock_coef, 2, range)

* Locate the Stocks with maximum and minimum parameters

In [ ]:
max(stock_coef[,"alpha1"])
which.max(stock_coef[,"alpha1"])
min(stock_coef[,"beta1"])
which.min(stock_coef[,"beta1"])

**<span style='background:yellow'>Stock1 GNW</span> has <span style='background:yellow'>Maximum $\alpha_1$</span> at 0.56, as well as minimum $\beta_1$ at 0.42.**

In [ ]:
max(stock_coef[,"beta1"])
which.max(stock_coef[,"beta1"])
min(stock_coef[,"alpha1"])
which.min(stock_coef[,"alpha1"])

**<span style='background:yellow'>Stock2 CHRW</span> has Minimum $\alpha_1$ at 0.004, as well as <span style='background:yellow'>Maximum $\beta_1$</span> at 0.98.**

In [ ]:
min(stock_coef[,"alpha1+beta1"])
which.min(stock_coef[,"alpha1+beta1"])
max(stock_coef[,"alpha1+beta1"])
which.max(stock_coef[,"alpha1+beta1"])


**<span style='background:yellow'>Stock3 PRGO</span>  has <span style='background:yellow'>Minimum $\alpha_1+\beta_1$</span> at 0.73, while <span style='background:yellow'>Stock4 PSX</span>  has <span style='background:yellow'>Maximum $\alpha_1+\beta_1$</span> at 0.99.**

In [ ]:
# Also check the highest Volatility stock
max(stock_coef[,"omega"])
which.max(stock_coef[,"omega"])

**<span style='background:yellow'>Stock5 BBBY</span>  has <span style='background:yellow'>Highest Volatility $\omega$</span> at 7.05.**

* Plot those stocks with extreme parameters for comparation

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 8)
par(mfrow = c(3, 2))

plot(lrets$GNW,  main = "GNW.Max Alpha1")
plot(lrets$CHRW,  main = "CHRW.Max Beta1")
plot(lrets$PRGO,  main = "PRGO.Min Alpha1+Beta1")
plot(lrets$PSX,  main = "PSX.Max Alpha1+Beta1")
plot(lrets$BBBY,  main = "BBBY.Max omega")

**Comments:  
From the chart, Stock4 PSX which has <span style='background:yellow'>Maximum $\alpha_1+\beta_1$</span> shows volatility spike persistence much longer, as well as much slower decay after spikes, and also shows the existing of volatility clustering.** 

* Plot the volatility chart for stocks with Maximum and minimum $\alpha_1+\beta_1$

In [ ]:
t_lrets <- merge(lrets$PRGO, lrets$PSX, all=TRUE)
autoplot(t_lrets, facets = FALSE) + geom_line() + theme_wsj()+ 
    scale_colour_wsj("dem_rep")+ ggtitle("Volatility Clustering Effects")

**<span style='background: lightblue'>Conclutions:</span>  
By comparing the volatility of <span style='color:red'>Stock PXS with Maximum $\alpha_1+\beta_1$</span> to <span style='color:blue'>Stock PRGO with Minimum $\alpha_1+\beta_1$</span>, we could clearly see PXS shows more volatility clustering Effects than PRGO.   
Although both stocks have zero-means, returns in blue line, tends to back to mean much faster then red line. The returns in red line obviously are not independent, after a large return change, another large change tends to follow in the next day. This pattern in the volatility clusters, shows the past shocks are persistent, negative shocks even persisted longer.** 

### 5. Use the GARCH volatility of the cross-section of 100 stocks for every day to obtain median "market" volatility and its quantiles (95% and 5%). 

* Observe stock volatility by chart

In [ ]:
par(bg = "#f7f7f7")
options(repr.plot.width = 8, repr.plot.height = 6)
plot(stock_coef[,"omega"])

* Apply GARCH result to calculate cross-section "market" volatility 

In [ ]:
median_vol <- median(stock_coef[,"omega"])
median_vol
quantile(stock_coef[,"omega"],c(0.05, 0.95))

**Coments:  
Our median "market" volatility is 0.207, 5% quantiles at 0.058, 95% quantiles at 1.456.**

### Create a figure plotting the cross-section of median, the 95% quantile, and the 5% quantile "market" volatility that you have obtained. Please, comment.

In [ ]:
par(bg = "#f7f7f7")
options(repr.plot.width = 10, repr.plot.height = 6)
par(mfrow = c(1, 2))
plot((1:N - 1)/(N - 1), sort(stock_coef[,"omega"]), type="l",
     main = "Cross-section Volatility Quantiles", xlab = "Fraction", ylab = "Quantile")

hist(stock_coef[,"omega"], n = 100, probability = TRUE, border = "white",
     col = "steelblue", xlab = NA, main = "Cross-section Market Volatility")

**Comments:  
By formula defination $$\sigma_t^2 = \omega + \alpha_1 a_{t - 1}^2 + \beta_1 \sigma_{t - 1}^2$$ 
we could see $\omega$ is the long-term volatility. From our observation of Cross-section volatility from GARCH model results, 95% of stocks has $\omega$ lower than 1.5.**

# Problem 2

### Use the dataset of logarithmic returns from the first problem. Compute the mean logarithmic return for each date. Estimate a suitable ARMA family model for this series. Consider at least three different ARMA orders that you can pinpoint as candidate models by observing the ACF and PACF. Remember to include all the necessary steps including model diagnostics. Discuss the results in detail.

### 1.  Compute the mean logarithmic return

In [ ]:
load("2-Returns.RData")

length(lrets)
names(lrets)
head(lrets[[1]])
tail(lrets[[1]])

**Comments:  
We need to manipulate the list in the way that all symbols have matching timestamps, so it is necessary to check if all time-serie observations are matching each other.**

In [ ]:
# Check time stamps
p_dates <- index(lrets[[1]])
length(p_dates)
for (i in 2:length(lrets)){
    p_dates <- index(lrets[[i]])[index(lrets[[i]]) %in% p_dates]
}
length(p_dates)

* Calculate mean logarithmic return for each date

In [ ]:
# Calculate the time serises of cross sector means
N <- nrow(lrets[[1]])  
lrets_mean <- sapply(1:N, function(y){
   mean(sapply(lrets, '[[', y)) 
})

length(lrets_mean)
head(lrets_mean)


In [ ]:
# Create an xts object
lrets_xts <- xts(lrets_mean, index(lrets[[1]]))
head(lrets_xts)

### 2. Estimate a suitable ARMA family model for this series. 

* (1). We start with plotting the time series shape.

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 6)
plot(lrets_xts,  xlab = 'Year', ylab = 'Percent', main = 'Mean Logarithmic Return', 
     axes = F, cex.main = 0.8)

**Comments:  
From the plot we could feel that the Mean is probably 0, there is clearly autocorrelations, and it should be stationary. And from the chart we could not see obvious jump or shift, so no need for subsample testing.   
Also we could observe when Covid started at 2020 spring, there is a spike of much higher volatility and the chart shows volatility clustering after. Also at Winter of 2020 when Delta Covid appeared, there was high volatility and persisit cluster.** 

* (2). Apply lag depenency test by exploring ACF and PACF

In [ ]:
par(bg = "#f7f7f7")
options(repr.plot.width = 8, repr.plot.height = 6)
par(mfrow = c(1, 2))
acf(lrets_xts, main = NA, lag = 50)
pacf(lrets_xts, main = NA, lag = 50)

**Comments:  
From ACF and PACF test, there seems a lot of dependence visible in first 8 lags, especially at lag 1 and 2, so <span style='background:yellow'>we choose AR(1), ARMA(1,1) and ARMA(2,1)</span> as candidate models.  
And we could not rule out stationarity based on ACF and PACF, still need to conduct a formal test for autocorrelation and stationarity.**

*  (3). Apply Ljun-Box test to check if this time series contains autocorrelation.

In [ ]:
Box.test(lrets_xts, type = 'Ljung-Box')
Box.test(lrets_xts, lag = 8, type = 'Ljung-Box')

**Comments:  
As we expected, since both Box-Ljung test for 1 lag and 8 lags have p_value < 0.01, so <span style='background:yellow'>this time series has autocorrelation</span>.**

* (4). Formal test for stationarity.

In [ ]:
adf.test(lrets_xts, k = 1)
adf.test(lrets_xts, k = 5)
adf.test(lrets_xts, k = 10)

kpss.test(lrets_xts, null = 'Level')
kpss.test(lrets_xts, null = 'Trend')

**Comments:  
Because in Augmented Dickey-Fuller Test all testing p-value results < 0.05, and in KPSS Test all testing p-value results > 0.05, so we could reject non-stationarity, and conclude that <span style='background:yellow'>this time series is stationary</span>.**

* (5). Apply Auto Arima Test

In [ ]:
lrets_auto <- auto.arima(lrets_xts, max.d = 0)
lrets_auto

par(bg = "#f7f7f7")
options(repr.plot.width = 8, repr.plot.height = 6)
par(mfrow = c(1, 2))

acf(lrets_auto$residuals, main = NA)
pacf(lrets_auto$residuals, main = NA)

**Comments:  
From ACF and PACF Test of residuals, We could see depenency at lag 1, 7, and 17.  
And auto.arima suggest ARMA(4,4).**

### 3. Consider at least three different ARMA orders that you can pinpoint as candidate models by observing the ACF and PACF. 

* (1). First we try AR(1) as starter

In [ ]:
# use Arima package to feed AR(1), (AR order, diff, MA roder)
lrets_ar1 <- Arima(lrets_xts, order = c(1, 0, 0))
summary(lrets_ar1)

plot(lrets_ar1$residuals, type = 'h', ylab = NA, main = 'Residuals')

par(mfrow = c(1, 2))
acf(lrets_ar1$residuals, main = 'Residuals - ACF')
pacf(lrets_ar1$residuals, main = 'Residuals - PACF')
acf(lrets_ar1$residuals ^ 2, main = 'Squared residuals - ACF')
pacf(lrets_ar1$residuals ^ 2, main = 'Squared residuals - PACF')


**Comments:  
Just see the coefficient at -0.17 and a huge standard error sigma^2 at 2.4, we would know  <span style='background:yellow'>AR(1) would not be the answer</span>.  Also there is many autocorrelation in residuals.**

* (2). Next, try if ARMA(1, 1) would be a better fit.

In [ ]:
# try ARMA(1,1), order is 1,0,1  (AR, Diff, MA)
lrets_arma11 <- Arima(lrets_xts, order = c(1, 0, 1))
summary(lrets_arma11)
plot(lrets_arma11$residuals, type = 'h', ylab = NA, main = 'Residuals')
par(mfrow = c(1, 2))
acf(lrets_arma11$residuals, main = 'Residuals - ACF')
pacf(lrets_arma11$residuals, main = 'Residuals - PACF')
acf(lrets_arma11$residuals ^ 2, main = 'Squared residuals - ACF')
pacf(lrets_arma11$residuals ^ 2, main = 'Squared residuals - PACF')



**Comments:  
Coefficient incresed to -0.52, but still huge standard error sigma^2 at 2.41, as well as many autocorrelation in residuals, so  <span style='background:yellow'>ARMA(1,1) clearly is not the good fit either</span>.**

In [ ]:
Box.test(lrets_arma11$residuals, type = "Ljung-Box", lag = 4)
Box.test(lrets_arma11$residuals, type = "Ljung-Box", lag = 8)
Box.test(lrets_arma11$residuals, type = "Ljung-Box", lag = 12)

**Just as we expected, Box-Ljung test for lag 8 and 12 have p_value < 0.01, the residuals have many autocorrelations.**

* (3). Try ARMA(2, 1)

In [ ]:
# try ARMA(2,1), order is 2,0,1  (AR, Diff, MA)
lrets_arma21 <- Arima(lrets_xts, order = c(2, 0, 1))
summary(lrets_arma21)
plot(lrets_arma21$residuals, type = 'h', ylab = NA, main = 'Residuals')
par(mfrow = c(1, 2))
acf(lrets_arma21$residuals, main = 'Residuals - ACF')
pacf(lrets_arma21$residuals, main = 'Residuals - PACF')
acf(lrets_arma21$residuals ^ 2, main = 'Squared residuals - ACF')
pacf(lrets_arma21$residuals ^ 2, main = 'Squared residuals - PACF')

**Comments:  
In ARMA(2,1) there are still some autocorrelation in residuals, so we do the LB test for residual dependency.**

In [ ]:
Box.test(lrets_arma21$residuals, type = "Ljung-Box")
Box.test(lrets_arma21$residuals, type = "Ljung-Box", lag = 3)
Box.test(lrets_arma21$residuals, type = "Ljung-Box", lag = 6)
Box.test(lrets_arma21$residuals, type = "Ljung-Box", lag = 8)

**Comments:  
From Box-Ljung test, seems lags 6 and 8 have p_value < 0.01, so there still are dependency in residuals. <span style='background:yellow'>ARMA(2,1) is still not the right answer</span>.**

* (4). Test ARIMA's suggestion of ARMA(4, 4)

In [ ]:
# try ARMA(4,4), order is 4,0,4  (AR, Diff, MA)
lrets_arma44 <- Arima(lrets_xts, order = c(4, 0, 4))
summary(lrets_arma44)
plot(lrets_arma44$residuals, type = 'h', ylab = NA, main = 'Residuals')
par(mfrow = c(1, 2))
acf(lrets_arma44$residuals, main = 'Residuals - ACF')
pacf(lrets_arma44$residuals, main = 'Residuals - PACF')
acf(lrets_arma44$residuals ^ 2, main = 'Squared residuals - ACF')
pacf(lrets_arma44$residuals ^ 2, main = 'Squared residuals - PACF')

**Comments:  
In ARMA(4,4) looks better, but seems still has some distant autocorrelation in residuals, this could caused by the ARCH effect. so we do the residual auto-correlation test.**

In [ ]:
Box.test(lrets_arma44$residuals, type = "Ljung-Box")
Box.test(lrets_arma44$residuals, type = "Ljung-Box", lag = 7)
Box.test(lrets_arma44$residuals, type = "Ljung-Box", lag = 10)
Box.test(lrets_arma44$residuals, type = "Ljung-Box", lag = 17)

**Comments:  
From Box-Ljung test, for lag 1, 8, 10 and 17 all have p_value > 0.01, so seems the residual autocorrelation is removed. So <span style='background:yellow'>ARMA(4,4) could be the right answer</span>.**

* Test for conditional heteroscedasticity in the residuals.

In [ ]:
lrets_44 <- arima(lrets_xts, order = c(4, 0, 4))

arch.test(lrets_44)

**Comments:  
We do not see the conditional heteroscedasticity in the residuals results.**

* (5). Compare models by Information Criteria

In [ ]:
models <- 4
criteria <- matrix(ncol = 2, nrow = models)
colnames(criteria) <- c('AIC', 'BIC')
rownames(criteria) <- c('AR(1)', 'ARMA(1,1)', 'ARMA(2,1)', 'ARMA(4,4)')
criteria[1, 1] <- lrets_ar1$aic
criteria[1, 2] <- lrets_ar1$bic
criteria[2, 1] <- lrets_arma11$aic
criteria[2, 2] <- lrets_arma11$bic
criteria[3, 1] <- lrets_arma21$aic
criteria[3, 2] <- lrets_arma21$bic
criteria[4, 1] <- lrets_arma44$aic
criteria[4, 2] <- lrets_arma44$bic
criteria

**<span style='background: lightblue'>Conclutions:</span>  
From Information Criteria, ARMA(4,4) has both lowest AIC and BIC, so <span style='background:yellow'>we select ARMA(4,4) as the best fit</span>.**